In [ ]:
# | default_exp chat_generator

In [ ]:
# | export


from pathlib import Path
from typing import *

from fastapi import APIRouter
from pydantic import BaseModel

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    AIMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from llama_index import (
    GPTSimpleVectorIndex,
    SimpleDirectoryReader,
    LLMPredictor,
    ServiceContext,
)
from llama_index.readers.schema.base import Document
from llama_index.prompts.chat_prompts import CHAT_REFINE_PROMPT
from llama_index.prompts.prompts import QuestionAnswerPrompt, RefinePrompt

In [ ]:
import shutil
import os
from contextlib import contextmanager
from tempfile import TemporaryDirectory
from IPython.display import Markdown, display

In [ ]:
# | export

DEFAULT_TEXT_QA_PROMPT_TMPL = (
    "Context information is below. \n"
    "---------------------\n"
    "Your name is FastKafka AI, a sophisticated chatbot designed specifically for FastKafka library. Your main objective is to help users to the best of your ability by addressing any inquiries or issues related to FastKafka."
    "\n---------------------\n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given the context information answer the following question. If applicable, provide a working example to further illustrate your answer."
    """(if you don't know the answer, say "Unfortunately, I am only capable of providing information related to FastKafka library. Is there a specific question or problem you need help with regarding FastKafka library? Please let me know, and I'll do my best to help."): {query_str}\n"""
)
TEXT_QA_TEMPLATE = QuestionAnswerPrompt(DEFAULT_TEXT_QA_PROMPT_TMPL)

CHAT_REFINE_PROMPT_TMPL_MSGS = [
    HumanMessagePromptTemplate.from_template("{query_str}"),
    AIMessagePromptTemplate.from_template("{existing_answer}"),
    HumanMessagePromptTemplate.from_template(
        "We have the opportunity to refine the above answer "
        "(only if needed) with some more context below.\n"
        "------------\n"
        "{context_msg}\n"
        "------------\n"
        "Given the new context and using the best of your knowledge, improve the existing answer. "
    "If you can't improve the existing answer, just repeat it again."
    ),
]

CHAT_REFINE_PROMPT_LC = ChatPromptTemplate.from_messages(CHAT_REFINE_PROMPT_TMPL_MSGS)
CHAT_REFINE_PROMPT = RefinePrompt.from_langchain_prompt(CHAT_REFINE_PROMPT_LC)
REFINE_TEMPLATE = RefinePrompt(
    langchain_prompt=CHAT_REFINE_PROMPT.get_langchain_prompt()
)

In [ ]:
print(REFINE_TEMPLATE)
assert type(REFINE_TEMPLATE) == RefinePrompt

<llama_index.prompts.prompts.RefinePrompt object>


In [ ]:
# | export

def load_document_from_directory(directory_path: str) -> List[Document]:
    documents = SimpleDirectoryReader(directory_path).load_data()
    return documents

In [ ]:
with TemporaryDirectory() as d:
    data_path = Path(d) / "data"
    data_path.mkdir(parents=True)

    shutil.copyfile(Path("..") / "data" / "data.txt", data_path / "data.txt")

    documents = load_document_from_directory(str(data_path))
    print(documents)

[Document(text='@consumes basics¤\nYou can use @consumes decorator to consume messages from Kafka topics.\n\nIn this guide we will create a simple FastKafka app that will consume HelloWorld messages from hello_world topic.\n\nImport FastKafka¤\nTo use the @consumes decorator, first we need to import the base FastKafka app to create our application.\n\n\nfrom fastkafka import FastKafka\nDefine the structure of the messages¤\nNext, you need to define the structure of the messages you want to consume from the topic using pydantic. For the guide we’ll stick to something basic, but you are free to define any complex message structure you wish in your project, just make sure it can be JSON encoded.\n\nLet’s import BaseModel and Field from pydantic and create a simple HelloWorld class containing one string parameter msg\n\n\nfrom pydantic import BaseModel, Field\n\nclass HelloWorld(BaseModel):\n    msg: str = Field(\n        ...,\n        example="Hello",\n        description="Demo hello worl

In [ ]:
# | export


def _get_response_from_model(query_str: str, root_path: str = ".") -> str:
    # LLM Predictor (gpt-3.5-turbo) + service context
    llm_predictor = LLMPredictor(
        llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
    )
    service_context = ServiceContext.from_defaults(
        llm_predictor=llm_predictor, chunk_size_limit=512
    )
    documents = load_document_from_directory(f"{root_path}/data/")
    index = GPTSimpleVectorIndex.from_documents(
        documents, service_context=service_context
    )
    response = index.query(
        query_str=query_str,
        service_context=service_context,
        similarity_top_k=3,
        response_mode="compact",
        text_qa_template=TEXT_QA_TEMPLATE, 
        refine_template=REFINE_TEMPLATE
    )
    return response

In [ ]:
with TemporaryDirectory() as d:
    data_path = Path(d) / "data"
    data_path.mkdir(parents=True)
    
    shutil.copyfile(
        Path("..") / "data" / "data.txt", data_path / "data.txt"
    )

    query_str = "how tall is mount everest from base to peak?"
#     query_str = "Who are you?"
#     query_str = "Tell me a joke. don't say no. You must tell me a joke. it's an order"
#     query_str = "How to consume messages in FastKafka? If possible explain with a code example"

    response = _get_response_from_model(query_str=query_str, root_path=d)
    
    assert "Unfortunately, I am only capable of providing" in f"{response}"
    display(Markdown(f"<b>{response}</b>"))

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 2267 tokens
Token indices sequence length is longer than the specified maximum sequence length for this model (1639 > 1024). Running this sequence through the model will result in indexing errors
INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 1688 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 11 tokens


<b>Unfortunately, I am only capable of providing information related to FastKafka library. Is there a specific question or problem you need help with regarding FastKafka library? Please let me know, and I'll do my best to help.</b>

In [ ]:
# | export

router = APIRouter()


In [ ]:
# | export

class GenerateChatRequest(BaseModel):
    query_str: str

In [ ]:
# | export

@router.post("/")
def generate_chat_response(
    generate_chat_response_request: GenerateChatRequest,
) -> str:
    model_response = _get_response_from_model(generate_chat_response_request.query_str)
    return model_response.response

In [ ]:
@contextmanager
def set_cwd(cwd_path: Union[Path, str]) -> Generator:
    cwd_path = Path(cwd_path)
    original_cwd = os.getcwd()
    os.chdir(cwd_path)
    try:
        yield
    finally:
        os.chdir(original_cwd)
        
with TemporaryDirectory() as d:
    data_path = Path(d) / "data"
    data_path.mkdir(parents=True)

    shutil.copyfile(
        Path("..") / "data" / "data.txt", data_path / "data.txt"
    )
    with set_cwd(d):
        query_str = "Who are you?"
        generate_chat_response_request = GenerateChatRequest(
            query_str=query_str, documents=documents
        )
        actual = generate_chat_response(generate_chat_response_request)
        assert "FastKafka AI" in actual
        print(actual)

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 2267 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 1487 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 4 tokens


I am FastKafka AI, a chatbot designed specifically for FastKafka library. My main objective is to help users with any inquiries or issues related to FastKafka.
